In [1]:
# import requests

# url = 'http://www.wunderground.com/robots.txt'
# response  = requests.get(url)
# print(response.text)

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time, os

In [3]:
#conda install -c conda-forge selenium

In [4]:
#pip install -U selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
driver = webdriver.Chrome('/usr/bin/chromedriver')

In [6]:
url = 'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2020-3'
driver.get(url)
time.sleep(1)

In [7]:
driver.page_source[:30]

'<html lang="en" itemscope="" i'

In [8]:
# url='https://www.wunderground.com/history/daily/us/ca/oakland/KOAK/date/2020-7-1'
# html_content = requests.get(url).text
soup = BeautifulSoup(driver.page_source, 'lxml')
#print(soup.prettify())

In [9]:
table = soup.find_all('table')
#print("Number of tables on site:", len(table))

In [10]:
Month = table[1]

In [11]:
head_line = Month.find("thead")
#print(head_line)
# Declare empty list to keep Columns names
headings = []
#for item in head.find_all("th"): # loop through all th elements
for item in head_line.find_all("td"): # find the first td element
    # convert the td elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['Time', 'Temperature (° F)', 'Dew Point (° F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (Hg)', 'Precipitation (in)']


In [12]:
# first_header = ['Time']
# last_header = ['Precipitation (in)']
# column_headers = ['Temperature (° F)', 'Dew Point (° F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (Hg)']
# sub_headers = ['_max', '_avg', '_min']
# new_headers = first_header
# for i in column_headers:
#     for j in sub_headers:
#         new_headers.append(i + j)
# new_headers.extend(last_header)
# print(new_headers)

In [13]:
data_set = [  [2020, 3, 31], [2020, 4, 30]
            , [2020, 5, 31], [2020, 6, 30], [2020, 7, 31], [2020, 8, 31]
            , [2020, 9, 30], [2020, 10, 31], [2020, 11, 30], [2020, 12, 31]
            , [2021, 1, 31], [2021, 2, 28], [2021, 3, 31], [2021, 4, 30]
            , [2021, 5, 31], [2021, 6, 30], [2021, 7, 31], [2021, 8, 31], [2021, 9, 30]  ]
            #, [2019, 1, 31], [2019, 2, 28],[2019, 3, 31], [2019, 4, 30]
            #, [2019, 5, 31], [2019, 6, 30], [2019, 7, 31], [2019, 8, 31]
            #, [2019, 9, 30], [2019, 10, 31], [2019, 11, 30], [2019, 12, 31]
            #, [2020, 1, 31], [2020, 2, 29], ]
#print(data_set)
header_set = False
for ls in data_set:
    c_url = 'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/' + str(ls[0]) +'-' + str(ls[1])
    #print(c_url)
    #url = 'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2020-3'
    driver.get(c_url)
    time.sleep(15)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    table = soup.find_all('table')
    Month = table[1]
    entries = ls[2] + 1

    body = Month.find("tbody") 
    # print(body)
    tr_tbody=body.find('tr')
    #print(combo_1)

    d = {'Date':[0] * entries, 'Temp_max':[0] * entries, 'Temp_avg':[0] * entries
             , 'Temp_min':[0] * entries, 'DP_max':[0] * entries, 'DP_avg':[0] * entries
             , 'DP_min':[0] * entries, 'Humid(%)_max':[0] * entries, 'Humid(%)_avg':[0] * entries
             , 'Humid(%)_min':[0] * entries, 'WindSp(mph)_max':[0] * entries, 'WindSp(mph)_avg':[0] * entries
             , 'WindSp(mph)_min':[0] * entries, 'Pr(Hg)_max':[0] * entries, 'Pr(Hg)_avg':[0] * entries
             , 'Pr(Hg)_min':[0] * entries, 'Precip(in)':[0] * entries}
    
    # for dct in d:
    #     for old_key, new_key in new_headers[0].items():
    #         dct[new_key] = dct[old_key]
    #         del dct[old_key]        
    df = pd.DataFrame(data=d)
    dfcol = 0
    tdnum = -1
    for td in tr_tbody:
        tdnum += 1  # Expect 0 through 7 
        #print('Outer-Td') 
        #print(tdnum)
        for table in td:
            trnum = 0
            #print('table')
            #print(trnum)
            for tr in table:
                #print('trs in table-in-td')
                #print(tdnum, trnum)

                if ((trnum == 0) or (trnum == entries)):
                    ; #print ('SKIP Tdnum 0')  
                else:
                    #print ('Update Tdnum/Trnum')  
                    #print(tdnum, trnum) 
                    #tds = tr.find('td')
                    inner_tdnum = -1
                    if tdnum == 0:
                        dfcol = tdnum
                    else:
                        dfcol = 1 + (tdnum -1) * 3 
                    for td in tr:
                        if (((tdnum == 0) and (inner_tdnum == 0)) or ((tdnum == 6) and (inner_tdnum == 0)) or  ((tdnum != 0) and (inner_tdnum == 2))):
                            ; #print('Skipping last td entry in tr') 
                        else:
                            inner_tdnum += 1
                            update_col = dfcol + inner_tdnum
                            #print('tdnum/trnum/inner_tdnum/dfcol/update_col')  #trnum or row_id?)
                            #print(tdnum, trnum, inner_tdnum, dfcol,update_col)
                            #print(td)
                            for item in td:#combo_1.find_all("td"): # find the first td element
                                item = (item).rstrip("\n").strip(" ")
                                #print(update_col, trnum, item)
                                df.iat[trnum, update_col] = item
                                #print(df)
                trnum += 1

    df['Year'] = ls[0]
    df['Month'] = ls[1]
    if(header_set == True):
        df1 = df.drop([0])
        result = final_df.append(df1)
        final_df = result
        #final_df.extend(df1) #+append - appropriate df something
        #print('Skipping 0 and 1')
        #print(header_set)
    else:
        final_df = df.drop([0])
        header_set = True
    

    #list = c_url.split("-")
    #list2 = list[-1]
    #print(list2)
    #df1['Month']=list2
    #print(final_df) #.head()
                       

    #print(td1)
    #     df = pd.DataFrame(data=td,columns=new_headers)
    #     df.head()
    #     print(td.text)
    #     print('\n')

    # Declare empty list to keep Columns names

    # td1=combo_1.find_all('td')
    # print(td1)
    # for d in range(31):
    #  d += 1
    #  list = url.split("-")
    #  list2 = list.remove([-1])
    #  url = list2[0] + '-7-' + str(d)

print(final_df.head(40))
final_df.to_csv('final_df1.csv', index = False)

   Date Temp_max Temp_avg Temp_min DP_max DP_avg DP_min Humid(%)_max  \
1     1       44     35.5       26     17   13.7     10           55   
2     2       56     48.2       38     38   30.5     17           73   
3     3       58     52.4       48     50   44.6     35           89   
4     4       57     50.5       46     37   28.5     19           66   
5     5       52     44.8       40     34   25.4     19           71   
6     6       45     41.6       37     36   34.0     30           89   
7     7       47     41.0       37     32   16.5      5           76   
8     8       60     47.8       37     21   14.7      9           41   
9     9       72     59.3       47     32   22.8     17           41   
10   10       66     61.0       55     49   40.2     30           64   
11   11       56     50.4       44     42   28.5     20           66   
12   12       48     44.7       42     41   36.1     22           85   
13   13       70     54.6       43     56   41.3     14         

In [186]:
# body = Mar20.find_all("tr")
# # Head values (Column names) are the first items of the body list
# head = body[1] # 0th item is the header row
# #head_2 = body[1] # 0th item is the header row
# body_rows = body[2:] # All other items becomes the rest of the rows

# # Lets now iterate through the head HTML code and make list of clean headings

# # Declare empty list to keep Columns names
# headings = []
# #for item in head.find_all("th"): # loop through all th elements
# for item in head.find_all("td"): # find the first td element
#     # convert the td elements to text and strip "\n"
#     item = (item.text).rstrip("\n")
#     # append the clean column name to headings
#     headings.append(item)
# print(headings)

In [187]:
# body_1 = Mar20.find_all("tr")
# # Head values (Column names) are the first items of the body list
# head_2 = body_1[1] # 0th item is the header row
# body_rows = body_1[2:] # All other items becomes the rest of the rows

# # Lets now iterate through the head HTML code and make list of clean headings

# # Declare empty list to keep Columns names
# headings_2 = []
# #for item in head.find_all("th"): # loop through all th elements
# for item in head_2.find_all("td"): # find the first td element
#     # convert the td elements to text and strip "\n"
#     item = (item.text).rstrip("\n")
#     # append the clean column name to headings
#     headings_2.append(item)
# print(headings_2)

In [188]:
# Next is now to loop though the rest of the rows

#print(body_rows[0])
# all_rows = [] # will be a list for list for all rows
# for row_num in range(31):#len(body_rows)): # A row at a time
#     row = [] # this will old entries for one row
#     for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
#         # row_item.text removes the tags from the entries
#         # the following regex is to remove \xa0 and \n and comma from row_item.text
#         # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
#         #row_item = (row_item.text).rstrip([])
#         aa = re.sub("(\xa0)|(\n)|,","",row_item.text).strip(" ")
#         #append aa to row - note one row entry is being appended
#         row.append(aa)
#     # append one row to all_rows
#     all_rows.append(row)
# print(all_rows)

In [189]:
# print(body_rows[0])
# all_rows = [] # will be a list for list for all rows
# for row_num in range([32:64]):#len(body_rows)): # A row at a time
#     row = [] # this will old entries for one row
#     for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
#         # row_item.text removes the tags from the entries
#         # the following regex is to remove \xa0 and \n and comma from row_item.text
#         # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
#         #row_item = (row_item.text).rstrip([])
#         aa = re.sub("(\xa0)|(\n)|,","",row_item.text).strip(" ")
#         #append aa to row - note one row entry is being appended
#         row.append(aa)
#     # append one row to all_rows
#     all_rows.append(row)
# print(all_rows)

In [190]:
# column_headers = ['Time', 'Temperature (° F)', 'Dew Point (° F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (Hg)', 'Precipitation (in)']
# sub_headers = ['_max', '_avg', '_min']
# new_headers = []
# for i in column_headers:
#     for j in sub_headers:
#         new_headers.append(i + j)
# print(new_headers)

In [15]:
driver.quit

<bound method WebDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="8a435217a849abb88c0a7d174d7bb1b3")>>

In [198]:
# df = pd.DataFrame(data=all_rows,columns=headings)
# df.head()

In [199]:
#driver.quit

In [200]:
# df['Date']='1'
# df.head()

In [201]:
# df['Year']=1

In [202]:
# df.head()

In [9]:
# ridership = pd.read_csv('MTA.csv')
# ridership.head()

,Date,Subways: Total Estimated Ridership,Subways: % Change From Pre-Pandemic Equivalent Day,Buses: Total Estimated Ridership,Buses: % Change From Pre-Pandemic Equivalent Day,LIRR: Total Estimated Ridership,LIRR: % Change From 2019 Monthly Weekday/Saturday/Sunday Average,Metro-North: Total Estimated Ridership,Metro-North: % Change From 2019 Monthly Weekday/Saturday/Sunday Average,Access-A-Ride: Total Scheduled Trips,Access-A-Ride: % Change From Pre-Pandemic Equivalent Day,Bridges and Tunnels: Total Traffic,Bridges and Tunnels: % Change From Pre-Pandemic Equivalent Day
0,9/23/2021,3019343,-50.2%,1382795,-41.6%,140100.0,-51%,115500.0,-56%,23031,-24.7%,915198,-6.0%
1,9/22/2021,3026655,-50.1%,1476326,-38.6%,139900.0,-51%,120100.0,-54%,23157,-27.2%,872883,-7.9%
2,9/21/2021,3024458,-49.7%,1490251,-37.5%,142700.0,-51%,123900.0,-52%,22399,-25.5%,860050,-6.8%
3,9/20/2021,2889141,-49.8%,1473044,-37.1%,148100.0,-49%,122100.0,-54%,21889,-21.9%,895109,-3.0%
4,9/19/2021,1598414,-38.9%,672307,-40.0%,69400.0,-26%,64100.0,-20%,12612,-32.6%,914623,-2.1%
